In [4]:
%load_ext autoreload
%autoreload 2

from google.colab import drive

drive.mount("/content/drive")

import plotly.express as px
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import seaborn as sns
from tqdm.notebook import trange, tqdm

import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers.schedules import InverseTimeDecay
from tensorflow.keras.optimizers import Adam

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error

# !pip install wandb -qqq
# import wandb
# from wandb.keras import WandbCallback
# wandb.login()

import sys

price_predictor = (
    "/content/drive/MyDrive/1 Projects/bitcoin_price_predictor/price_predictor"
)
sys.path.append(price_predictor)

from helpers import (
    load_close_data,
    get_n_test_samples,
    repeat_evaluate,
    summarize_scores,
    train_and_validate,
)

DOWNLOAD_DIR = Path("../download")
DOWNLOAD_DIR = Path(
    "/content/drive/MyDrive/1 Projects/bitcoin_price_predictor/download"
)
DATA_DIR = Path("/content/drive/MyDrive/1 Projects/bitcoin_price_predictor/data")

sns.set()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install wandb -qqq
import wandb
from wandb.keras import WandbCallback

wandb.login(key="9c152286056d5f0799f7da8912b364988c902f58")

wandb: Currently logged in as: theadammurphy (use `wandb login --relogin` to force relogin)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publically.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
data = load_close_data(DOWNLOAD_DIR, dropna=True)
data.shape

(92192,)

# Manual LR Scheduling

In [12]:
for i in range(5):
    print(f"\nRUN NUMBER {i}\n")
    run = wandb.init(
        project="bitcoin_price_predictor",
        config={
            ### Notebook environment
            "notebook": "colab",  # colab or local
            ### Data preparation
            "dataset": 1,
            "scaler": "log_and_range_0_1",  #  log, log_and_divide_a, log_and_range_a_b
            "n_input": 300,  # num lag observations
            ### Model build
            "model_type": "LSTM_SMALL",
            "activation": "relu",
            "loss": "mse",
            "optimizer": "adam",  # adam, rmsprop
            #### LR scheduler and optimizer
            "use_lr_scheduler": False,
            #                         'lr_scheduler': 'custom', # InverseTimeDecay, ExponentialDecay, cusom
            #                         'initial_lr': 1e-4,
            "lr": 1,  # if use_lr_scheduler == False for fixed LR
            ### Model fit
            "n_epochs": 50,  # num training epochs
            "n_batch": 168 * 9,  # batch size
            "verbose": 1,  # verbosity of  fit
            ### EarlyStopping callback
            "patience": 10,
            "restore_best_weights": True,
            "early_stopping_baseline": None,  # set to None if there isn't one
            # Plots
            "start_plotting_epoch": 10,
        },
    )
    config = wandb.config  # we use this to configure our experiment

    history = train_and_validate(config)
run.finish()


RUN NUMBER 0



TypeError: ignored